In [30]:
import pandas as pd
import praw
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
from praw.models import MoreComments

In [7]:
# define unique user agent, as per reddit API rule guidelines
user_agent = "windows:mdsa:v1.0 (by /u/datasci_guy)"

# initiate reddit instance
reddit = praw.Reddit(user_agent= user_agent,
                     client_id="P3uTH6wSXQJID55D18Y9jQ", client_secret="9rkhOaTSzcuybjAXgRlAW8mdv5swgQ")

print(reddit.read_only)

True


### Obtain a Subreddit

In [9]:
# define subreddit of interest
subreddit = reddit.subreddit("Calgary")

# display subreddit information
print(subreddit.display_name)
print(subreddit.title)
print(subreddit.description)

Calgary
Calgary: Onward
#### [View all /r/Calgary comments](https://www.reddit.com/r/Calgary/comments)

**Welcome to Calgary**

Please [send a message to the mods](http://www.reddit.com/message/compose?to=%2Fr%2FCalgary) if you have any questions or concerns. 


* http://en.wikipedia.org/wiki/Calgary
* http://www.calgary.ca
* http://www.visitcalgary.com
* http://www.calgarystampede.com

-

>##Our Rules
**Please also read/follow: [reddiquette](https://www.reddithelp.com/hc/en-us/articles/205926439)**
>
1. **Be respectful to each other.** Our goal for /r/Calgary is a friendly community where residents can engage in thoughtful discussion. Personal attacks will not be tolerated. No insults, bigotry, excessive foul language or trolling. No threats. Please be civil. Moderators will remove comments at their discretion. Note: Abuse of moderators will not be tolerated.
2. **No spam.** Generally, no selling goods or services. No self-promotion.  No fundraising. This includes Kijiji/Craigslist/Fa

### Obtain submission instance from subreddit

In [14]:
for submission in subreddit.hot(limit=10): # pulls the first 10 submissions from the "hot" tab of our subreddit
    print(submission.title)
    print(submission.score)
    print(submission.id)
    print(submission.url)

Moving To Calgary Megathread - January 2023 Edition
76
100hdhe
https://www.reddit.com/r/Calgary/comments/100hdhe/moving_to_calgary_megathread_january_2023_edition/
Want to learn more about Calgary's iconic Chinatown? Of course you do! Join us on Wednesday March 8th at 7:00pm for a live AMA with an expert on this community and ask all your questions!
46
11ldcjy
https://www.reddit.com/r/Calgary/comments/11ldcjy/want_to_learn_more_about_calgarys_iconic/
I got slapped by a crackhead this morning
339
11m3mj3
https://www.reddit.com/r/Calgary/comments/11m3mj3/i_got_slapped_by_a_crackhead_this_morning/
My last timelapse from the Feb 26-27th G2/G3 solar storm that covered Calgary in aurora all night. As we approach Solar maximum July 2025 here’s hoping there are more nights like these!
13
11m7i09
https://v.redd.it/z3dcm97hbmma1
Condo move in fee?
24
11m1fml
https://www.reddit.com/r/Calgary/comments/11m1fml/condo_move_in_fee/
A new Netflix series will be filmed in Alberta this summer
21
11m2ch6


In [17]:
# submissions instances can be called directly
submission = reddit.submission("100hdhe")

print(submission.title)
print(submission.score)
print(submission.id)
print(submission.url)

Moving To Calgary Megathread - January 2023 Edition
71
100hdhe
https://www.reddit.com/r/Calgary/comments/100hdhe/moving_to_calgary_megathread_january_2023_edition/


### Determine Available Attributes of an Object

In [24]:
submission = reddit.submission("100hdhe")

# displays all possible attributes of an object  
pprint.pprint(vars(submission))


{'_comments_by_id': {},
 '_fetched': False,
 '_reddit': <praw.reddit.Reddit object at 0x0000013EF643D9A0>,
 'comment_limit': 2048,
 'comment_sort': 'confidence',
 'id': '100hdhe'}


### Comment Extraction

In [27]:
for top_level_comments in submission.comments:
    print(top_level_comments.body)

I am a 32 year old gay male, corporate professional working remotely as a Marketing Director and am thinking of making the move. While I am fortunate to be financially comfortable and free in Vancouver (where I’ve lived for the last 9 years) I just want my money to go further. I’ve been comparing what I could rent for an apartment in Calgary vs. Vancouver and it blows my mind. With a budget around 3k a month I could get a really interesting and large space. 

Is there a strong queer scene in Calgary? What areas would anyone recommend? For anyone who knows Vancouver, I currently live in Gastown and love the aesthetic - cobble stones, old architecture, and an overall quaint-artsy vibe. I also LOVE lofts and was curious to know if any areas in Calgary are known for them. Any insight is appreciated!!!
26, Male, electrician/teledata tech in IBEW

Hello all, I'd like to discuss the current situation of moving to Calgary as a current American citizen. Hopefully without too much hostility as i

In [ ]:
from praw.models import MoreComments # load nested comments?

for top_level_comment in submission.comments:
    if isinstance(top_level_comment, MoreComments):
        continue
    print(top_level_comment.body)

In [ ]:
headlines = set()
headline_id = set()

for submission in reddit.subreddit('calgary').hot(limit=None): # hot, top, new, rising
    #print(submission.title)
    #print(submission.score)
    #print(submission.upvote_ratio)
    #print(submission.id)
    #print(submission.url)
    #print(submission.upvote_ratio)
    headlines.add(submission.title)
    headline_id.add(submission.id)


print(len(headlines))

In [33]:
# Convert comments to dataframe
headline_df = pd.DataFrame(headlines)
headline_df.head()

,0
0,Best Local Grocery Store
1,Families fear overcrowding as UCP funds only t...
2,Data shows how much time Canadians spent in ru...
3,"Kind of an unrelated post, but has anyone take..."
4,Sometimes you gotta stay up later than anticip...


In [34]:
headline_df.to_csv("headlines.csv", header=False, encoding='utf-8', index=False)

In [35]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jorda\AppData\Roaming\nltk_data...


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for line in headlines:
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)

#pprint(results[:3], width=100)
results


In [41]:
df = pd.DataFrame.from_records(results)
df.head()

,neg,neu,pos,compound,headline
0,0.000,0.417,0.583,0.6369,Best Local Grocery Store
1,0.242,0.758,0.000,-0.4939,Families fear overcrowding as UCP funds only t...
2,0.000,0.802,0.198,0.6369,Data shows how much time Canadians spent in ru...
3,0.000,0.823,0.177,0.6486,"Kind of an unrelated post, but has anyone take..."
4,0.000,1.000,0.000,0.0000,Sometimes you gotta stay up later than anticip...
